Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [40]:
batch_size = 128
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size), name="train_dataset")
    tf_train_label = tf.placeholder(tf.float32, shape=(batch_size,num_labels), name="label_dataset")
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases) 
    #logits = tf.matmul(tf_train_dataset, weights) + biases 
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_label)) + 0.01 * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases))
    test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases))


In [39]:
# Without regualizaton
import random
num_steps = 20000
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    for step in xrange(num_steps):
        offset = random.randint(0,len(test_labels) - batch_size)
        train_batch = train_dataset[offset: offset + batch_size, :] 
        labels_train_batch = train_labels[offset: offset + batch_size, :]
        feed_dict = {tf_train_dataset : train_batch, tf_train_label : labels_train_batch}
        _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, labels_train_batch))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
         

Minibatch loss at step 0: 17.447813
Minibatch accuracy: 13.3%
Validation accuracy: 14.9%
Minibatch loss at step 1000: 1.535544
Minibatch accuracy: 42.2%
Validation accuracy: 54.2%
Minibatch loss at step 2000: 1.493140
Minibatch accuracy: 53.9%
Validation accuracy: 70.7%
Minibatch loss at step 3000: 1.014021
Minibatch accuracy: 65.6%
Validation accuracy: 72.8%
Minibatch loss at step 4000: 1.268205
Minibatch accuracy: 56.2%
Validation accuracy: 73.3%
Minibatch loss at step 5000: 1.059533
Minibatch accuracy: 63.3%
Validation accuracy: 73.8%
Minibatch loss at step 6000: 1.144282
Minibatch accuracy: 57.0%
Validation accuracy: 74.2%
Minibatch loss at step 7000: 0.934217
Minibatch accuracy: 64.8%
Validation accuracy: 74.3%
Minibatch loss at step 8000: 1.166713
Minibatch accuracy: 54.7%
Validation accuracy: 74.6%
Minibatch loss at step 9000: 1.102345
Minibatch accuracy: 59.4%
Validation accuracy: 74.6%
Minibatch loss at step 10000: 0.984490
Minibatch accuracy: 61.7%
Validation accuracy: 74.9%


KeyboardInterrupt: 

<b>Accuracy - Without relu, without normalization</b>

Validation accuracy: 76.2%
Test accuracy: 83.5%

<b>Accuracy - With relu, without normalization</b>

Validation accuracy: 78.5%
Test accuracy: 85.9%

<b>Accuracy - With relu, with L2 normalization, alpha = 0.01</b>

Validation accuracy: 81.5%
Test accuracy: 88.5%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
